In [1]:
!nvidia-smi

Sat Jul  2 19:15:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%capture
!pip install datasets
!pip install transformers
!pip install rouge-score
!pip install nltk
!pip install sentencepiece

In [3]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from datasets import load_dataset, load_metric
import torch, random
import nltk
from nltk import sent_tokenize

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
rouge = load_metric("rouge", seed=2022)

In [7]:
model_checkpoint = 'google/pegasus-xsum'

In [8]:
tokenizer = PegasusTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

In [9]:
model = PegasusForConditionalGeneration.from_pretrained(model_checkpoint)
model.to(device)

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(512, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): Laye

In [10]:
data_test = load_dataset('xsum', split='test')

Using custom data configuration default


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934. Subsequent calls will reuse this data.


In [11]:
batch_size = 8

In [12]:
# map data correclty
def generate_summary(batch):
    inputs = tokenizer(batch['document'],
                       padding=True,
                       truncation=True,
                       max_length=512,
                       add_special_tokens=False,
                       return_tensors='pt')
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # The following decoding parameters are set according to config.json
    outputs = model.generate(input_ids,
                             attention_mask=attention_mask,
                             max_length=64, # config
                             num_beams=8, # config
                             length_penalty=0.6 # config
                             )

    # all special tokens will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch['prediction'] = output_str

    return batch

In [13]:
results = data_test.map(generate_summary,
                        batched=True,
                        batch_size=batch_size,
                        remove_columns=['document', 'id'])

labels = results['summary']
predictions = results['prediction']

Parameter 'function'=<function generate_summary at 0x7fb581d8d9e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1417 [00:00<?, ?ba/s]

In [19]:
# ROUGE expects a newline after each sentence
clean_preds = ["\n".join(sent_tokenize(pred.replace('<n>', '\n'))) for pred in predictions]
clean_labels = [label.replace(" .", ".") for label in labels]

for i in range(len(predictions)):
    print(f"Item {i}:")
    print(f"Ground truth: {clean_labels[i]}")
    print(f"Prediction: {clean_preds[i]}")
    print("\n")

Streaming output truncated to the last 5000 lines.


Item 9721:
Ground truth: Former Labour minister Kim Howells has said the party is in the "deepest crisis" he can remember.
Prediction: Labour's general election defeat was "at least as bad as under Michael Foot's leadership", former shadow Welsh secretary David Howells has said.


Item 9722:
Ground truth: Former England winger Ugo Monye believes Denny Solomona has a good chance of playing for the national team when he becomes eligible.
Prediction: Alan Solomona is "good enough" to play for England, says former England winger Ugo Monye.


Item 9723:
Ground truth: Weaknesses in Police Scotland's roll-out of its new national call-handling system have been highlighted in an Inspector of Constabulary report.
Prediction: A "damning indictment" of Police Scotland's call-handling project has been published by the chief inspector of Constabulary.


Item 9724:
Ground truth: Mayoral candidate, friend to the stars, charity fundraiser, lover of p

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
rouge_output = rouge.compute(predictions=clean_preds, references=clean_labels, use_stemmer=True)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(rouge_output[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)

print(rouge_dict)

{'rouge1': 46.95, 'rouge2': 24.45, 'rougeL': 39.08, 'rougeLsum': 39.11}


In [20]:
# map data correclty
def generate_summary(batch):
    inputs = tokenizer(batch['document'],
                       padding=True,
                       truncation=True,
                       max_length=512,
                       add_special_tokens=False,
                       return_tensors='pt')
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # The following decoding parameters are set according to config.json
    outputs = model.generate(input_ids,
                             attention_mask=attention_mask,
                             min_length=11, 
                             max_length=64, # config
                             num_beams=8, # config
                             length_penalty=0.6 # config
                             )

    # all special tokens will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch['prediction'] = output_str

    return batch

In [ ]:
results = data_test.map(generate_summary,
                        batched=True,
                        batch_size=batch_size,
                        remove_columns=['document', 'id'])

labels = results['summary']
predictions = results['prediction']

  0%|          | 0/1417 [00:00<?, ?ba/s]

In [ ]:
# ROUGE expects a newline after each sentence
clean_preds = ["\n".join(sent_tokenize(pred.replace('<n>', '\n'))) for pred in predictions]
clean_labels = [label.replace(" .", ".") for label in labels]

for i in range(len(predictions)):
    print(f"Item {i}:")
    print(f"Ground truth: {clean_labels[i]}")
    print(f"Prediction: {clean_preds[i]}")
    print("\n")